## Семинар 8: "Современные модели для NLP"

ФИО:

### На семинаре мы разберем [код трансфомера на pytorch](https://nlp.seas.harvard.edu/2018/04/03/attention.html)

###  ДЗ [3 балла]

Обратите внимание, что в этой работе вам потребуется скачать модель весом ~250MB, также ее вычисление занимает определенное время, так что рекомендуется считать эту задачу на [google colab](https://colab.research.google.com/).

In [ ]:
import torch
!pip install transformers
from transformers import *

     |████████████████████████████████| 1.4MB 6.1MB/s 
     |████████████████████████████████| 890kB 48.1MB/s 
     |████████████████████████████████| 2.9MB 24.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b6528ce52c7a2f386033115b3146dd562978e80cdb7f6d681354e2c68f13cff9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
MODEL = (DistilBertForMaskedLM, DistilBertTokenizer, 'distilbert-base-cased')

model_class, tokenizer_class, pretrained_weights = MODEL
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
input_ids = tokenizer.encode("Here is some text to encode", add_special_tokens=True)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
print(input_ids)

[101, 3446, 1110, 1199, 3087, 1106, 4035, 13775, 102]


In [ ]:
tokenizer.decode(input_ids)

'[CLS] Here is some text to encode [SEP]'

In [ ]:
input_ids[4] = tokenizer.mask_token_id
tokenizer.decode(input_ids)

'[CLS] Here is some [MASK] to encode [SEP]'

In [ ]:
input_batch = torch.tensor(input_ids).unsqueeze(0) # batch_size 1
with torch.no_grad():
    res = model(input_batch)[0]

In [ ]:
prob = torch.nn.functional.softmax(res, dim=-1)
new_ids = prob.max(-1)[1]

In [ ]:
tokenizer.decode(new_ids.numpy()[0, :].tolist())

'. here is some way to encode.'

In [1]:
GPT_TEXTS = [
    "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.",
    "A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown."
    ]

Ваша задача - сгенерировать продолжение текстов, на которых демонстрировалась работа GPT-2 с помощью загруженной модели (DistillBERT). Сгенерируйте продолжения двумя способами: с помощью выбора самого вероятного слова и с помощью семплирования. Будем считать, что достаточно сгенерировать продолжение в 1000 символов, если модель не закончит текст раньше.

Генерировать тексты дистиллированным бертом не получилось, поэтому я взял BERT из pytorch и попробовал генерировать с помощью него, получилось чуть лучше.

In [2]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 5.3MB/s 
     |████████████████████████████████| 102kB 4.8MB/s 
     |████████████████████████████████| 7.0MB 5.5MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
  Created wheel for boto3: filename=boto3-1.16.31-py2.py3-none-any.whl size=128450 sha256=80dcdf81ff521b3fe3bea5447b5bb461d57393ec85d1199edcfd582697d1cf08
  Stored in directory: /root/.cache/pip/wheels/d7/49/8d/c7b244e2b89911f7eb4d8db3b2ec5700b443e280fa68b6f9a0
Successfully built boto3
ERROR: botocore 1.19.31 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [3]:
import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from tqdm import tqdm

In [4]:
model_version = 'bert-base-uncased'
model = BertForMaskedLM.from_pretrained(model_version)
model = model.cuda()
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=model_version.endswith("uncased"))

def tokenize(text):
    return tokenizer.convert_tokens_to_ids(text)

def untokenize(tokens):
    return tokenizer.convert_ids_to_tokens(tokens)
    
def wordpiece2sentence(tokens):
    new_tokens = []
    for token in tokens:
        if token.startswith("##"):
            new_tokens[-1] += token[2:]
        else:
            new_tokens.append(token)
    return new_tokens

CLS_TOKEN = '[CLS]'
SEP_TOKEN = '[SEP]'
MASK_TOKEN = '[MASK]'
MASK_ID = tokenizer.convert_tokens_to_ids([MASK_TOKEN])[0]
SEP_ID = tokenizer.convert_tokens_to_ids([SEP_TOKEN])[0]
CLS_ID = tokenizer.convert_tokens_to_ids([CLS_TOKEN])[0]

100%|██████████| 231508/231508 [00:00<00:00, 304576.71B/s]


In [5]:
def get_index_by_probs(probs, index, temperature=1, top_k=0):
    # probs.shape = (1, seq_len, vocab_size)
    logits = probs[:, index]
    logits /= temperature
    
    if top_k > 0:
        kth_values, kth_indices = logits.topk(top_k, dim=-1)
        dist = torch.distributions.categorical.Categorical(logits=kth_values)
        return int(kth_indices.gather(dim=1, index=dist.sample().unsqueeze(-1)).squeeze(-1))
    
    elif top_k == 0:
        dist = torch.distributions.categorical.Categorical(logits=logits)
        return int(dist.sample().squeeze(-1))
    
    else:
        return int(torch.argmax(logits, dim=-1))

In [9]:
def get_init_text(seed_text, length):
    batch = seed_text + [MASK_TOKEN] * length + [SEP_TOKEN]
    return tokenize(batch)

def generate_random_position(seed_text, length=15, top_k=0,
                                   temperature=1, n_iters=300, burnin=200):
    seed_length = len(seed_text)
    batch = get_init_text(seed_text, length)
    
    for iter in range(n_iters):
        position = np.random.randint(0, length)
        batch[seed_length + position] = MASK_ID

        input = torch.tensor(batch).cuda().unsqueeze(0)
        with torch.no_grad():
            output = model(input)
        topk = top_k if (iter >= burnin) else 0
        index = get_index_by_probs(output, index=seed_length + position, top_k=topk,
                             temperature=temperature)
        batch[seed_length + position] = index

    return untokenize(batch)

            
def generate_sequential(seed_text, length=15, out_length=15, 
                          top_k=0, temperature=1):
    seed_length = len(seed_text)
    batch = get_init_text(seed_text, length)
    
    for iter in range(length):
        input = batch[:seed_length + iter + out_length]+[SEP_ID]
        input = torch.tensor(batch).cuda().unsqueeze(0)
        with torch.no_grad():
            output = model(input)
        index = get_index_by_probs(output, index=seed_length + iter, top_k=top_k,
                             temperature=temperature)
        batch[seed_length + iter] = index
        
    return untokenize(batch)


def generate(seed_text=CLS_TOKEN, length=25, top_k=100,
             temperature=1, burnin=200, n_iters=500, generator_mode="random",
             out_length=15):
    """if top_k = -1: max mode
       if top_k = 0: sample_mode
       if top_k > 0: topk mode
    """

    if generator_mode == "random":
        batch = generate_random_position(seed_text, length=length,
                                               top_k=top_k, temperature=temperature,
                                               burnin=burnin, n_iters=n_iters)
    elif generator_mode == "seq":
        batch = generate_sequential(seed_text, length=length, top_k=top_k,
                                      temperature=temperature,
                                      out_length=out_length)
    
    return batch

Основной проблемой BERT для генерации текстов является то, что при обучении ему был доступен контекст слова как слева, так и справа. Если генерировать текст слева направо (фукнция `generate_sequential`), то кажется, что он будет решать нетипичную для него задачу. Поэтому таке была реализована вторая функция `generate_random_position`, которая на каждой итерации генерирует слово для случайной позиции в области масок ([CLS], [SEED_TEXT], [MASK], [MASK], ...,[MASK], [SEP]). Через несколько итераций накапливаются слова. Какое-то из этих слов на текущей итерации может быть закрыто маской, и для этой позиции будет сгененировано новое слово. Но так как эта позиция чаще всего будет находиться "в глубине" предложения, окруженная контекстом других сгенерированных слов, то для генерации данного слова модель будет находиться в более "привычных" условиях.

burnin - количество итераций, в течение которых делается простое семлирование очередного слова. Когда количество итераций превышает burnin, делаем семплирование среди top-k слов по вероятности.

In [12]:
def generate_by_seed(seed_string, params):
    seed_text = (CLS_TOKEN + " " + seed_string.lower().replace(".", " .").replace(",", " ,")).split()
    output_text = generate(seed_text=seed_text, **params)
    return " ".join(wordpiece2sentence(output_text)[1:-1])

In [16]:
params = {
    "length": 20,
    "top_k": 1000,
    "temperature": 1,
    "out_length": 5,
    "burnin": 250,
    "n_iters": 500,
    "generator_mode": "random"
}

generate_by_seed(GPT_TEXTS[1], params)

'a train carriage containing controlled nuclear materials was stolen in cincinnati today . its whereabouts are unknown . information was obtained by the smithfield test bureau and the atomic energy administration from each united states official .'

In [ ]:
params = {
    "length": 20,
    "top_k": 1000,
    "temperature": 1,
    "out_length": 5,
    "burnin": 250,
    "n_iters": 500,
    "generator_mode": "seq"
}
generate_by_seed(GPT_TEXTS[1], params)

'a train carriage containing controlled nuclear materials was stolen in cincinnati today . its whereabouts are unknown . the stolen nuclear - power reactor was stolen from a cnc chemical reactor in neutral a paris province .'

#### Feedback (опционально)

Здесь вы можете оставить список опечаток из лекции или семинара:

Здесь вы можете оставить комментарии по лекции или семинару: